In [1]:
import vk_api
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np

In [2]:
def get_friends_graph(group_id, access_token):
    """
    Parse a graph of subscribers of a particular group on Vk.com. 
    Edge from i to j is conducted if i is in friends of j.
    In order to get access to VK api one needs access_token from standalone app
    
    Variable offset needs to shift a selection for parsing.
    
    Returns graph, number of users who are not been parsed because of private or deleted account
    and total number of members in group
    """
    vk_session = vk_api.VkApi(token=access_token)
    vk = vk_session.get_api()
    private_users = 0
    offset = -1
    graph = nx.Graph()
    followers = np.array([])
    
    get_followers = vk.groups.getMembers(group_id=group_id, offset = offset+1)
    number_members = get_followers['count']
    followers = np.append(followers, get_followers['items'])
    proxy = number_members - len(get_followers['items'])
    offset += 1000
    
    while  proxy != 0:
        get_followers = vk.groups.getMembers(group_id=group_id, offset = offset)
        proxy = len(get_followers['items'])
        offset += 1000
        followers = np.append(followers, get_followers['items'])
 
    
    
    for follower in followers:
        graph.add_node(follower)

    for follower in followers:
        try:
            friends = vk.friends.get(user_id=follower)
        except vk_api.exceptions.ApiError:
            private_users += 1
            friends = {'items':[]}
        else: pass
        for friend in friends['items']:
            if friend in followers:
                graph.add_edge(follower, friend)
        
    print(f'Не удалось собрать информацию о: {private_users}')
    print(f'Всего найдено подписчиков: {number_members}')

    return graph, private_users, number_members